In [37]:
import numpy as np
import keras
from matplotlib import pyplot as plt
import os

In [128]:
# use const for weights

def array2c(array,name):
    temp = array.flatten(order='C')
    size = array.size
    count = 0
    s = 'float ' + name + '[' + str(size) + '] = \n{'
    for i in range(size):
        s += "{:.16f}".format(temp[i]) + ','
        count += 1
        if (count)%3 is 0:
            s += '\n'
    s += '}; \n'
    return s    
    

In [190]:
def weights2c(model,file):
    for layer in model.layers[1:]:
        weights = layer.get_weights()
        if 'dense' in layer.name:
            outshp = layer.output_shape[1:]
            inshp = layer.input_shape[1:]
            A = weights[0]
            b = weights[1]
            outrows = outshp[0]
            inrows = inshp[0]
            insize = np.prod(inshp)
            outsize = np.prod(outshp)
            if len(outshp)>1:
                outcols = outshp[1]
            else:
                outcols=1
            if len(inshp)>1:
                incols = inshp[1]
            else:
                incols=1
            s = 'size_t ' + layer.name + '_outrows =' + str(outrows) + ';\n'
            s += 'size_t ' + layer.name + '_outcols =' + str(outcols) + ';\n'
            s += 'size_t ' + layer.name + '_innerdim =' + str(incols) + ';\n'
            s += 'float ' + layer.name + '_output[' + str(outsize) + '] = {0}; \n \n'
        
            file.write(s)
            file.write(array2c(A,layer.name + '_kernel'))
            file.write(array2c(b,layer.name + '_bias'))
            file.write('\n \n')

        elif 'lstm' in layer.name:
            W = weights[0]
            U = weights[1]
            b = weights[2]
            
            units = layer.get_config()['units']
            inshp = layer.input_shape[1:]
            outshp = layer.output_shape[1:]
            outrows = outshp[0]
            inrows = inshp[0]
            insize = np.prod(inshp)
            outsize = np.prod(outshp)
            if len(outshp)>1:
                outcols = outshp[1]
            else:
                outcols=1
            if len(inshp)>1:
                incols = inshp[1]
            else:
                incols=1
                
            Wi = W[:, :units]
            Wf = W[:, units: units * 2]
            Wc = W[:, units * 2: units * 3]
            Wo = W[:, units * 3:]

            Ui = U[:, :units]
            Uf = U[:, units: units * 2]
            Uc = U[:, units * 2: units * 3]
            Uo = U[:, units * 3:]

            bi = b[:units]
            bf = b[units: units * 2]
            bc = b[units * 2: units * 3]
            bo = b[units * 3:]
            
            s = 'size_t ' + layer.name + '_units = ' + str(units) + ';\n'
            s += 'size_t ' + layer.name + '_in_height = ' + str(inrows) + ';\n'
            s += 'size_t ' + layer.name + '_in_width = ' + str(incols) + ';\n'
            s += 'float ' + layer.name + '_xi[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_xf[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_xc[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_xo[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_yi[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_yf[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_yc[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_yo[' + str(units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_states[' + str(2*units) + '] = {0}; \n'
            s += 'float ' + layer.name + '_output[' + str(units) + '] = {0}; \n \n'
            file.write(s)
            
            file.write(array2c(Wi,layer.name + '_Wi'))
            file.write(array2c(Wf,layer.name + '_Wf'))
            file.write(array2c(Wc,layer.name + '_Wc'))
            file.write(array2c(Wo,layer.name + '_Wo'))

            file.write(array2c(Ui,layer.name + '_Ui'))
            file.write(array2c(Uf,layer.name + '_Uf'))
            file.write(array2c(Uc,layer.name + '_Uc'))
            file.write(array2c(Uo,layer.name + '_Uo'))

            file.write(array2c(bi,layer.name + '_bi'))
            file.write(array2c(bf,layer.name + '_bf'))
            file.write(array2c(bc,layer.name + '_bc'))
            file.write(array2c(bo,layer.name + '_bo'))
            file.write('\n \n')
            
        elif 'conv1d' in layer.name:
            filters = weights[0]
            bias = weights[1]
            inshp = layer.input_shape[1:]
            outshp = layer.output_shape[1:]
            outrows = outshp[0]
            inrows = inshp[0]
            insize = np.prod(inshp)
            outsize = np.prod(outshp)
            if len(outshp)>1:
                outcols = outshp[1]
            else:
                outcols=1
            if len(inshp)>1:
                incols = inshp[1]
            else:
                incols=1
            pad = layer.get_config()['padding']
            stride = layer.get_config()['strides'][0]
            dilation = layer.get_config()['dilation_rate'][0]
            kernel_size = layer.get_config()['kernel_size'][0]
            num_filters = layer.get_config()['filters']
            
            if pad in 'causal':
                pad_along_height = dilation*(kernel_size-1)
                pad_top = pad_along_height
                pad_bottom = 0
            elif pad in 'same':
                pad_along_height = max((outshp[0] - 1) * stride*dilation +
                        kernel_size - inshp[0], 0)
                pad_top = int(pad_along_height // 2)
                pad_bottom = int(pad_along_height - pad_top)
            elif pad in 'valid':
                pad_top=0
                pad_bottom=0
            
            s = 'size_t ' + layer.name + '_stride = ' + str(stride) + '; \n'
            s += 'size_t ' + layer.name + '_dilation = ' + str(dilation) + '; \n'
            s += 'size_t ' + layer.name + '_pad_top = ' + str(pad_top) + '; \n'
            s += 'size_t ' + layer.name + '_pad_bottom = ' + str(pad_bottom) + '; \n'
            s += 'size_t ' + layer.name + '_in_height = ' + str(inrows) + '; \n'
            s += 'size_t ' + layer.name + '_padded_in_height = ' + str(inrows + pad_top + pad_bottom) + '; \n'
            s += 'size_t ' + layer.name + '_in_width = ' + str(incols) + '; \n'
            s += 'size_t ' + layer.name + '_out_height = ' + str(outrows) + '; \n'
            s += 'size_t ' + layer.name + '_out_width = ' + str(outcols) + '; \n'
            s += 'size_t ' + layer.name + '_kernel_size = ' + str(kernel_size) + '; \n'
            s += 'float ' + layer.name + '_padded_input[' + str((inrows + pad_top + pad_bottom)*incols) + \
                '] = {0}; \n'
            s += 'float ' + layer.name + '_fill[' + str(incols) + '] = {0}; \n'
            s += 'float ' + layer.name + '_output[' + str(outsize) + '] = {0}; \n \n'
            file.write(s)
            file.write(array2c(filters, layer.name + '_kernel'))
            file.write(array2c(bias, layer.name + '_bias'))
            file.write('\n \n')
            
            

In [171]:
def layers2c(model,file):
    for i in range(1,len(model.layers)):
        layer = model.layers[i]
        prevlayer = model.layers[i-1]
        nm = layer.name
        pnm = prevlayer.name
        inputs = pnm + '_output'
        outputs = nm + '_output'
        if i is 1:
            inputs = 'inputs'
        if i is len(model.layers)-1:
            outputs = 'predictions'
                   
        if 'dense' in nm:
            activation = layer.get_config()['activation']
            if activation in 'tanh':
                activation = 'arrtanh'
            s = 'dense(' + outputs + ',' + inputs + ',' + nm + '_kernel, \n\t' + \
                   nm + '_bias,' + nm + '_outrows,' + nm + '_outcols, \n\t' + \
                   nm + '_innerdim,' + activation + '); \n'
            file.write(s)
        elif 'lstm' in nm:
            output_activation = layer.get_config()['activation']
            if output_activation in 'tanh':
                output_activation = 'arrtanh'
            recurrent_activation = layer.get_config()['recurrent_activation']
            if recurrent_activation in 'tanh':
                recurrent_activation = 'arrtanh'
            
            s = 'lstm(' + inputs + ',' + nm + '_states,' + nm + '_Wi, \n\t' + \
                   nm + '_Wf,' + nm + '_Wc,' + nm + '_Wo, \n\t' + \
                   nm + '_bi,' + nm + '_bf,' + nm + '_bc, \n\t' + \
                   nm + '_bo,' + nm + '_Ui,' + nm + '_Uf, \n\t' + \
                   nm + '_Uc,' + nm + '_Uo,' + nm + '_units, \n\t' + \
                   nm + '_in_height,' + nm + '_in_width,' + nm + '_xi, \n\t' + \
                   nm + '_xf,' + nm + '_xc,' + nm + '_xo, \n\t' + \
                   nm + '_yi,' + nm + '_yf,' + nm + '_yc, \n\t' + \
                   nm + '_yo,' + recurrent_activation + ',' + output_activation + ', \n\t' + \
                   outputs + '); \n'
            file.write(s)
        elif 'conv1d' in nm:
            activation = layer.get_config()['activation']
            if activation in 'tanh':
                activation = 'arrtanh'
            s = 'padding1d(' + inputs + ',' + nm + '_padded_input,' + nm + '_fill, \n\t' + \
                    nm + '_in_height,' + nm + '_in_width,' + nm + '_pad_top,' + nm + '_pad_bottom); \n'
            file.write(s)
            s = 'convolution1d(' + nm + '_padded_input,' + outputs + ',' + nm + '_kernel, \n\t' + \
               nm + '_bias,' + nm + '_out_height,' + nm + '_out_width, \n\t' + \
               nm + '_kernel_size,' + nm + '_padded_in_height,' + nm + '_in_width,' + nm + '_stride, \n\t' + \
               nm + '_dilation,' + activation + '); \n'
            file.write(s)

In [172]:
def model2c(model,file,function_name):
    s = '#include <stdio.h> \n #include <stddef.h> \n #include <math.h> \n'
    s += '#include "keras2c_include.h" \n'
    s += '\n \n'
    file.write(s)
    s = 'void ' + function_name + '(float inputs[], float predictions[]) { \n \n'
    file.write(s)
    weights2c(model,file)
    layers2c(model,file)
    file.write('\n }')

In [218]:
def keras2c(model_filepath,function_name,num_tests=10):

    function_name = str(function_name)
    filename = function_name + '.c'
    model = keras.models.load_model(str(model_filepath))
    file = open(filename,"w+")
    model2c(model,file,function_name)
    file.close()
    make_test_suite(model,function_name,num_tests)

In [222]:
def make_test_suite(model,function_name,num_tests=10):
    
    input_size = np.prod(model.layers[0].input_shape[1:])
    output_size = np.prod(model.layers[-1].output_shape[1:])
    
    file = open(function_name + '_test_suite.c',"w+")
    s = '#include <stdio.h> \n#include <math.h> \n#include <sys/time.h> \n \n'
    file.write(s)
    s = 'void ' + function_name + '(float input[],float prediction[]);\n' 
    s += 'float norm2(float array1[], float array2[], size_t numel);\n'
    s += 'struct timeval GetTimeStamp(); \n \n'
    file.write(s)
    s = 'int main(){\n'
    file.write(s)
    for i in range(num_tests):
        #generate random input and write to file
        rand_input = np.random.random(model.layers[0].input_shape[1:])
        file.write(array2c(rand_input,'rand_input' + str(i+1)))
        rand_input = rand_input[np.newaxis,...]
        # make predictions
        output = model.predict(rand_input)
        output = output[0,:]
        outshp = output.shape
        # write predictions
        file.write(array2c(output,'keras_output' + str(i+1)))
        s = 'float coutput' + str(i+1) + '[' + str(output_size) + '] = {0};\n'
        file.write(s)
    s = ' float errors[' + str(num_tests) + '];\n'
    s += ' size_t numel = ' + str(output_size) + ';\n'
    s += ' int num_tests = ' + str(num_tests) + '; \n'
    s += ' struct timeval t1 = GetTimeStamp(); \n'
    file.write(s)
    for i in range(num_tests):
        s = function_name + '(rand_input' + str(i+1) + ',' + 'coutput' + str(i+1) +'); \n'
        file.write(s)
    file.write('\n')
    s =  'struct timeval t2 = GetTimeStamp(); \n'
    s += 'typedef unsigned long long u64; \n'
    s += 'u64 t1u = t1.tv_sec*1e6 + t1.tv_usec; \n'
    s += 'u64 t2u = t2.tv_sec*1e6 + t2.tv_usec; \n'
    s += 'printf("time (us) %llu \\n", (t2u-t1u)/' + str(num_tests) + '); \n'
    file.write(s)
    for i in range(num_tests):
        s = 'errors[' + str(i) + '] = norm2(keras_output' + str(i+1) + ',coutput' + str(i+1) + ',numel); \n'
        file.write(s)
    s = 'printf("L2 norm of output errors for tests:\\n");\n'
    file.write(s)
    s = 'for(int i=0; i<' + str(num_tests) + ';i++){ \n'
    s += ' printf("test %d : %e \\n",i, errors[i]);}\n'
    file.write(s)
    file.write('return 0;\n } \n\n')
    s = """float norm2(float array1[], float array2[], size_t numel){ \n
    float sum = 0; \n
    for(size_t i=0; i<numel; i++){\n
    sum += (array1[i]-array2[i])*(array1[i]-array2[i]);}\n
    return sqrt(sum);}\n\n"""
    file.write(s)
    s = """struct timeval GetTimeStamp() {
    struct timeval tv;
    gettimeofday(&tv,NULL);
    return tv;
}"""
    file.write(s)
    file.close()

In [223]:
inshape = (8,23,)
stride=1
dilation=1
filter_height=3
num_filters=5
pad = 'valid'
a = keras.layers.Input(inshape)
b = keras.layers.Dense(48)(a)
c = keras.layers.Conv1D(filters=num_filters, kernel_size=filter_height, strides=stride, padding=pad, \
                        dilation_rate=dilation)(b)
d = keras.layers.LSTM(20)(c)
# e = keras.layers.Dense(30)(d)
#f = keras.layers.add([e,c])

model = keras.models.Model(inputs=a, outputs=d)
model.save('test1.h5')

In [224]:
keras2c('test1.h5','test1',100)

In [189]:
model.layers[3].output_shape

(None, 20)

In [161]:
a = np.random.random(model.layers[0].input_shape[1:])
a = a[np.newaxis,...]
b = model.predict(a)
b.shape

(1, 30)

In [137]:
a = np.random.random((4,3,2))
a.shape

(4, 3, 2)

In [138]:
a[1,:].shape

(3, 2)

In [139]:
a[1,:,:]-a[1,:]

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [163]:
c = np.array([[1,2,3,4],[5,6,7,8]])
c

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [164]:
c.flatten(order='C')

array([1, 2, 3, 4, 5, 6, 7, 8])